In [22]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,GlobalAveragePooling1D,Lambda,Bidirectional
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras import backend as B

# Load preprocessed data

In [23]:
q1_data = np.load(open('q1_train.npy', 'rb'))
q2_data = np.load(open('q2_train.npy', 'rb'))

labels = np.load(open('label_train.npy', 'rb'))
embedding_matrix = np.load(open('word_embedding_matrix.npy', 'rb'))

In [24]:
from sklearn.cross_validation import train_test_split

X = np.stack((q1_data, q2_data), axis=1)
target = labels

X_train, X_val, y_train, y_val = train_test_split(X, target, test_size=0.25, random_state=126, stratify=target)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_val = X_val[:,0]
Q2_val = X_val[:,1]

# set up model


In [25]:
def vec_distance(vects):
    x, y = vects
    return B.sum(B.square(x - y), axis=1, keepdims=True)
#don't use squar root of the sum, it doens't give a good range to feed to the dense layer.

In [26]:
def vec_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


In [27]:
from keras.layers.embeddings import Embedding

nb_words=95602+1
max_sentence_len=25
embedding_layer = Embedding(nb_words,300,
        weights=[embedding_matrix],
        input_length=max_sentence_len,trainable=False)
#dont train this layer!

In [28]:
lstm_layer =LSTM(128)

sequence_1_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

distance=Lambda(vec_distance, output_shape=vec_output_shape)([x1, y1])
dense1=Dense(16, activation='sigmoid')(distance)
dense1 = Dropout(0.3)(dense1)

bn2 = BatchNormalization()(dense1)
prediction=Dense(1, activation='sigmoid')(bn2)

model = Model(input=[sequence_1_input, sequence_2_input], output=prediction)

/home/ami/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


In [29]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 25)            0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 25)            0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 25, 300)       28680900    input_3[0][0]                    
                                                                   input_4[0][0]                    
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 128)           219648      embedding_2[0][0]       

In [30]:
#according to Keras, RMSprop (adaptive LR) is good for recurrent neural net. 
# Adam is another method that computes adaptive learning rates for each parameter. 
#In addition to storing an exponentially decaying average of past squared gradients vtvt like Adadelta and RMSprop, Adam also keeps an exponentially decaying average of past gradients mtmt,

###RMSprop as well divides the learning rate by an exponentially decaying average of squared gradients. 
##Adam is computationally efficient, has little memory requirements, is invariant to diagonal rescaling of the gradients, 
#and is well suited for problems that are large in terms of data and/or parameter
model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['acc'])

In [31]:

early_stopping =EarlyStopping(monitor='val_loss', patience=3)


In [32]:
#optional: try calculating class weights
#source: stack exchange, J.Guillaumin

import math

# labels_dict : {ind_label: count_label}
# mu : parameter to tune 

def create_class_weight(labels_dict,mu=2):
    total = np.sum(labels_dict.values())
    keys = labels_dict.keys()
    class_weight = dict()

    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0

    return class_weight


In [33]:
####optional, assign weights to the labels due to imbalanced labels (0,1)
unique, counts = np.unique(target, return_counts=True)
labels_dict=dict(zip(unique, counts))

target_weight=create_class_weight(labels_dict)
target_weight

{0: 1.1575228972468952, 1: 1.683427250535681}

In [34]:
hist=model.fit([Q1_train, Q2_train], y_train, validation_data=([Q1_val, Q2_val], y_val), verbose=1, 
          nb_epoch=10, batch_size=256, shuffle=True,class_weight=None, callbacks=[early_stopping])
#takes long time to initiate
#using dense() layer and sigmoid activation

Train on 257726 samples, validate on 85909 samples
Epoch 1/10
257726/257726 [==============================] - 897s - loss: 0.5562 - acc: 0.7138 - val_loss: 0.4447 - val_acc: 0.7960
Epoch 2/10
257726/257726 [==============================] - 895s - loss: 0.4392 - acc: 0.7978 - val_loss: 0.4135 - val_acc: 0.8107
Epoch 3/10
257726/257726 [==============================] - 897s - loss: 0.3973 - acc: 0.8248 - val_loss: 0.3909 - val_acc: 0.8243
Epoch 4/10
257726/257726 [==============================] - 898s - loss: 0.3663 - acc: 0.8429 - val_loss: 0.3805 - val_acc: 0.8304
Epoch 5/10
257726/257726 [==============================] - 898s - loss: 0.3412 - acc: 0.8571 - val_loss: 0.3787 - val_acc: 0.8349
Epoch 6/10
257726/257726 [==============================] - 897s - loss: 0.3188 - acc: 0.8694 - val_loss: 0.3734 - val_acc: 0.8377
Epoch 7/10
257726/257726 [==============================] - 898s - loss: 0.2990 - acc: 0.8792 - val_loss: 0.3753 - val_acc: 0.8370
Epoch 8/10
257726/257726 [======

/home/ami/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


# Save model and model weights

In [35]:
from keras.models import model_from_json

In [36]:
# export model to JSON
model_json = model.to_json()
with open("brnn_model_distance_128_d16_d05.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("brnn_model_distance_128_d16_d05.h5")
print("Saved model to disk")

Saved model to disk


In [37]:
# load json and create model
#json_file = open('brnn_model_distance_128_d16_d05.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#model = model_from_json(loaded_model_json)
# load weights into new model
#model.load_weights("brnn_model_distance_128_d16_d05.h5")
#print("Loaded model from disk")
 

In [38]:
test1_data = np.load(open('test1.npy', 'rb'))
test2_data = np.load(open('test2.npy', 'rb'))

In [39]:
pred=model.predict([test1_data, test2_data],verbose=1)


60640/60643 [============================>.] - ETA: 0s

In [40]:
submission=pd.read_csv('sample.csv')
print pred.clip(1e-5, 0.99999)


[[  8.73683544e-04]
 [  5.60724795e-01]
 [  7.94184627e-04]
 ..., 
 [  6.32060096e-02]
 [  7.53800210e-04]
 [  9.24243748e-01]]


In [41]:
submission['is_duplicate']=pred.clip(1e-5, 0.99999)
submission.to_csv('lstm_submission(13).csv', index=False)

In [42]:
#import zipfile

In [43]:
#with zipfile.ZipFile('lstm_submission(13).zip', 'w') as myzip:
#    myzip.write('lstm_submission(13).csv')